In [2]:
from dotenv import load_dotenv
import loggingService

from genai.extensions.langchain import LangChainInterface
from genai.model import Credentials, Model
from genai.schemas import GenerateParams

from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import Weaviate

import os
from sentence_transformers import SentenceTransformer
import weaviate

load_dotenv()
logger = loggingService.get_logger()


In [3]:
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", 'https://workbench-api.res.ibm.com')
# class_name = os.getenv("WEVIATE_CLASS", 'LivrosVectorizer')
print(f"API Key: {api_key}, API Endpoint: {api_endpoint}")
class_name ='Livros'
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question", output_key='answer', return_messages=True)
model_name = os.getenv('MODEL_NAME', 'bigscience/mt0-xxl')
model_name_embedding = os.getenv("MODEL_NAME_EMBEDDING", "sentence-transformers/gtr-t5-large")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(url=weaviate_url,)
embeddings = SentenceTransformer(model_name_embedding)

creds = Credentials(api_key, api_endpoint=api_endpoint)
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=100,
    min_new_tokens=1,
    stream=False,
    temperature=0.5,
    top_k=50,
    top_p=1,
).dict()  # Langchain uses dictionaries to pass kwargs

llm = LangChainInterface(model=model_name, credentials=creds, params=params)
vector_store = Weaviate(client=client, index_name='Livros', text_key='content')


API Key: pak-cHWs0qIzvsnI96qDydNytkNBaBXscYF0T3uPBR-2msE, API Endpoint: https://workbench-api.res.ibm.com
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [4]:
# vector_store.similarity_search_by_vector(embeddings.encode('por que arthur dent foi despejado?'))
vector_store.similarity_search('por que arthur dent foi despejado?')


[Document(page_content='\xa0\xa0\xa0Ford ouviu um leve gemido. À luz do fósforo, viu umaforma pesada mexendo-se no chão. Rapidamente apagouo fósforo, pôs a mão no bolso, encontrou o que procuravae tirou-o do bolso. Abriu o pacote e sacudiu-o. Ajoelhou-se. A forma mexeu-se de novo. Ford Prefect disse:\xa0 \xa0 \xa0 — Eu trouxe uns amendoins.\xa0 \xa0 \xa0 Arthur Dent mexeu-se e gemeu de novo, produzindosons incoerentes.\xa0 \xa0 \xa0 — Tome, coma um pouco — insistiu Ford, sacudindo opacote.\xa0 \xa0 \xa0 — Se você nunca passou antes por um raio detransferência de matéria, você deve ter perdido sal eproteína. A cerveja que você tomou deve ter protegidoum pouco seu organismo.\xa0 \xa0 \xa0 — Rrrrr... — disse Arthur Dent. Abriu os olhos. — Estáescuro.\xa0 \xa0 \xa0 — É — disse Ford Prefect — , está escuro, sim.\xa0 \xa0 \xa0 — Luz nenhuma — disse Arthur Dent. — Escuro,completamente. Uma das coisas que Ford Prefect jamaisconseguiu entender em relação aos seres humanos eraseu hábito de aﬁrma

In [5]:
pt1 = """Responda a pergunta a seguir de forma sucinta usando o contexto e histórico fornecidos. Caso não tenha certeza da resposta siceramente diga que não possui informações suficientes sobre esse tema.

Contexto: {context}

Histórico: {chat_history}

Pergunta: {question}
Resposta:"""
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=pt1,
)

retriver = vector_store.as_retriever(search_kwargs={'score_threshold': 0.8})
# chain_type_kwargs={"prompt": prompt},
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriver, memory=memory,  verbose=True)
# qa = ConversationalRetrievalChain(llm=llm, retriever=retriver, memory=memory,  verbose=True, output_key='Resposta')


In [6]:
qa.run('por que arthur dent foi despejado?')



> Entering new RetrievalQA chain...

> Finished chain.


KeyError: 'question'